In [1]:
!pip install torch torchvision yolov5 opencv-python matplotlib

import torch
import cv2
import os
import time
import numpy as np
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import torchvision.models as models
import torch.nn as nn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.5/953.5 kB 26.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 26.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.0/887.0 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 83.8 MB/s eta 0:00:00:00:010:01
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114248 sha256=a5dfbd38721f7ffa837eccfaa027b30913d7545cefcb5fff09baf6e72d0542a5
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire
  Attempting uni

In [2]:
# Load pre-trained YOLO model (YOLOv5)
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -qr requirements.txt  # Install dependencies
from yolov5 import detect

# Load YOLOv5s (small version) or a different YOLOv5 model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Cloning into 'yolov5'...
remote: Enumerating objects: 17045, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17045 (delta 10), reused 18 (delta 8), pack-reused 17022 (from 1)
Receiving objects: 100% (17045/17045), 15.67 MiB | 13.91 MiB/s, done.
Resolving deltas: 100% (11704/11704), done.
/kaggle/working/yolov5
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-11-18 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 125MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [4]:
# Load custom trained classifier for pedestrian types
def load_custom_trained_model(model_path):
    # Load the pre-trained ResNet18 model
    model = models.resnet18(pretrained=True)
    
    # Modify the fully connected layer as per the custom trained structure
    num_features = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(p=0.15),  # 15% dropout rate
        nn.Linear(num_features, 3)  # Assuming 3 output classes
    )
    
    # Load the state dictionary into the model
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    model.load_state_dict(state_dict)
    
    # Set the model to evaluation mode
    model.eval()
    
    return model

# Load the custom trained classifier
classifier_model = load_custom_trained_model('/kaggle/input/model-2/Model_2/fine_tuned_resnet18.pth')

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 61.0MB/s]


In [5]:
# Transformation for the classifier input
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust size if needed
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [31]:
# Import necessary libraries
import time
import cv2
import torch
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

# Define relevant classes for YOLO
relevant_classes = ['person', 'cat', 'dog', 'car', 'truck', 'bus', 'traffic light', 'traffic sign', 'bicycle', 'motorcycle']

# Function to process a folder of images
def detect_and_classify_folder(folder_path, output_folder):
    start_time = time.time()
    image_files = list(Path(folder_path).glob("*.png"))

    # Check if output folder exists, create if not
    Path(output_folder).mkdir(parents=True, exist_ok=True)

    for image_path in image_files:
        print(f"Processing {image_path.name}")  # Debugging step

        # Step 1: Run YOLOv5 on the image to detect objects
        results = yolo_model(str(image_path))
        detections = results.pandas().xyxy[0]
        print(detections)  # Debugging step to see if detections are returned

        # Read and prepare the image
        image = cv2.imread(str(image_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        original_image = image.copy()

 # Step 2: Iterate through detected bounding boxes for relevant classes
        for _, row in detections.iterrows():
            if row['name'] in relevant_classes:
                if row['name'] == 'person':
                    # Extract and adjust the bounding box coordinates
                    x_min, y_min, x_max, y_max = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
                    box_width = x_max - x_min
                    x_min = max(0, x_min - int(0.1 * box_width))
                    x_max = min(image.shape[1], x_max + int(0.1 * box_width))
                    y_max = min(image.shape[0], y_max + int(0.2 * (y_max - y_min)))
        
                    # Crop and classify the pedestrian or e-scooter rider
                    cropped_image = image[y_min:y_max, x_min:x_max]
                    pil_image = Image.fromarray(cropped_image)
                    input_tensor = transform(pil_image).unsqueeze(0)
        
                    with torch.no_grad():
                        outputs = classifier_model(input_tensor)
                        _, predicted = outputs.max(1)
        
                    # Label mapping for the classifier
                    labels = ['Pedestrian', 'E-scooter Rider', 'Pedestrian']
                    label = labels[predicted.item()]  # Assign either Pedestrian or E-scooter Rider
        
                    # Annotate the original image
                    cv2.rectangle(original_image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
                    cv2.putText(original_image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
        
                elif row['name'] == 'bicycle':
                    # If bicycle is detected, check if it is a bicycle rider using the classifier
                    x_min, y_min, x_max, y_max = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
                    
                    # Increase the y_min to include the torso of the rider (adjust the top boundary)
                    y_min = max(0, y_min - int(0.2 * (y_max - y_min)))  # Increase the top boundary
                    
                    # Crop and classify the bicycle rider (or just a bicycle)
                    cropped_image = image[y_min:y_max, x_min:x_max]
                    pil_image = Image.fromarray(cropped_image)
                    input_tensor = transform(pil_image).unsqueeze(0)
                    
                    with torch.no_grad():
                        outputs = classifier_model(input_tensor)
                        _, predicted = outputs.max(1)
                    
                    # Check if the classifier predicts a Bicycle Rider
                    if predicted.item() == 2:  # Assuming 'Bicycle Rider' is class 2 in labels
                        # If classifier confirms it as a Bicycle Rider, adjust the label
                        label = 'Bicycle Rider'
                    else:
                        # If classifier does not confirm it as a Bicycle Rider, it's a parked bicycle
                        label = 'Parked Bicycle'
                
                    # Annotate the original image with the updated bounding box and label
                    cv2.rectangle(original_image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                    cv2.putText(original_image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                else:
                    # For non-pedestrian, non-bicycle, relevant classes, annotate as detected
                    x_min, y_min, x_max, y_max = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
                    label = row['name']
                    cv2.rectangle(original_image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
                    cv2.putText(original_image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the image with annotations
        plt.figure(figsize=(10, 10))
        plt.imshow(original_image)
        plt.axis('off')
        plt.title(f'Detections for {image_path.name}')
        plt.show()

        # Save the annotated image
        output_path = f"{output_folder}/{image_path.name}"
        cv2.imwrite(output_path, cv2.cvtColor(original_image, cv2.COLOR_RGB2BGR))
        print(f"Image saved to {output_path}")

    # Print total time taken for inference
    end_time = time.time()
    print(f"Total inference time for {len(image_files)} images: {end_time - start_time:.2f} seconds")

# Example call to the function
# Replace '/path/to/input/folder' with your input folder path
detect_and_classify_folder('/kaggle/input/test-data1/Test_Images', '/kaggle/working')

Processing front_left_1717445646180.png
           xmin        ymin         xmax         ymax  confidence  class  \
0    769.933777  323.386658  1144.087891   647.227234    0.857940      7   
1   1287.127319   61.342827  1317.969482   141.290451    0.821731      9   
2   1502.468750  338.544922  1588.175537   490.379639    0.685731      0   
3    408.422241  126.711716   439.799011   193.475601    0.670473      9   
4    719.481201  378.137329   776.823364   414.088531    0.608702      2   
5   1342.014771  390.384064  1368.514404   469.925537    0.472879     10   
6    871.872437  218.379776   887.543884   256.938629    0.462804      9   
7    469.856934  150.505875   496.362732   209.063873    0.444711      9   
8     72.623047  181.437988   137.544296   236.600800    0.416652      9   
9      0.000000  747.792603  1709.910156  1080.000000    0.369070      2   
10  1491.424927  263.473450  1516.891724   300.506470    0.252452      9   

             name  
0           truck  
1   tra

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Image saved to /kaggle/working/front_left_1717445646180.png
Processing front_left_1718139279003.png
           xmin        ymin         xmax        ymax  confidence  class  \
0    898.392151  349.661743  1189.912720  474.208862    0.916360      2   
1    331.651703  381.987366   489.158936  466.057678    0.842881      2   
2    762.108154  369.151733   807.255066  410.935669    0.639147      2   
3    673.661438   16.929319   707.143921  104.030251    0.531611      9   
4   1471.527100  347.308990  1504.145020  426.060242    0.501823      0   
5    227.699188  210.406891   274.171936  269.324921    0.438817      9   
6   1466.172729  230.690140  1484.234619  263.362732    0.381615      9   
7   1124.991089  188.267853  1160.383911  242.041718    0.366712      9   
8     59.867477  396.464600    97.748306  457.075806    0.312364      0   
9    907.628540  249.129608   924.440186  277.865448    0.301471      9   
10  1626.372437  360.377289  1718.556885  401.038422    0.299056      2   


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Image saved to /kaggle/working/front_left_1718139279003.png
Processing front_left_1717445543053.png
           xmin        ymin         xmax         ymax  confidence  class  \
0   1553.274902  361.514038  1758.619995   431.075623    0.868556      2   
1     35.998875    2.281383    73.582718    79.501236    0.868546      9   
2   1414.182617  356.105988  1543.595581   422.508545    0.813764      2   
3    794.247070  329.212341  1114.522705   611.294617    0.804440      7   
4    932.364258  194.193268   947.809570   230.041473    0.718059      9   
5   1346.158813  213.511414  1384.694092   304.646393    0.708837      9   
6   1151.617432  341.196411  1420.566895   439.926208    0.700337      2   
7    856.297119  185.536194   872.911743   227.843719    0.681122      9   
8    156.058014  339.936462   487.263153   450.937195    0.632493      2   
9    399.434875  220.409042   422.762695   266.123596    0.607295      9   
10  1048.674316  360.144226  1164.417480   428.240540    0.60054

...

Image saved to /kaggle/working/front_left_1717445868057.png
Processing front_left_1717446418136.png
           xmin        ymin         xmax         ymax  confidence  class  \
0    621.694214  381.091095   777.261108   500.982880    0.870512      2   
1    400.625427  381.231537   571.344177   485.274628    0.868561      2   
2    883.592102  395.915466   980.305298   475.376526    0.813758      2   
3   1594.455811  356.858185  1917.673584   625.679077    0.768884      7   
4      0.000000  426.005219   346.393066   932.781250    0.768270      2   
5    770.160400  389.171509   832.285645   444.800598    0.747660      2   
6   1087.343750  393.096954  1179.925537   449.672150    0.692550      2   
7    315.878662  378.242310   403.284760   423.510559    0.558775      2   
8      0.000000  745.639709  1920.000000  1065.024170    0.473263      2   
9    209.013672  368.781982   307.964752   433.631042    0.329892      2   
10  1564.964722  411.912842  1642.599976   453.007446    0.32553

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Image saved to /kaggle/working/front_left_1717446418136.png
Processing front_left_1718139391143.png
          xmin        ymin         xmax        ymax  confidence  class  \
0   848.810791  350.365631  1057.915771  545.371765    0.831899      2   
1   487.224243  368.028076   604.968018  417.334625    0.733706      2   
2   742.223389  342.527924   864.973877  417.319702    0.684715      2   
3  1312.952148  334.883301  1384.648438  497.733276    0.660452      0   
4  1382.232056    3.764751  1449.135498  112.280869    0.536775      9   
5  1230.551758  355.163269  1284.649536  409.588165    0.528400      2   
6   429.776123  205.489075   447.874268  245.089172    0.371188      9   
7   835.402588  350.488129   889.193115  418.250793    0.368354      2   
8   488.685608  217.448242   505.481232  253.942688    0.362309      9   
9   354.916534  365.325989   376.384369  424.049744    0.312568      0   

            name  
0            car  
1            car  
2            car  
3        

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Image saved to /kaggle/working/front_left_1718139391143.png
Total inference time for 60 images: 16.29 seconds
